# Normalizace nad abecedou $\{0,1,2\}$

In [1]:
import gpc # library for work with GPS words over {0,1}
import math
import itertools
import re

## Work with $E_i$-palindromes and closures

In [2]:
def Ei(i):
    i = int(i)
    ei = [0,0,0]
    ei[i] = str(i)
    ei[(i+1)%3] = str((2+i)%3)
    ei[(i+2)%3] = str((1+i)%3)
    return ei

In [3]:
# Example:
print(Ei(1))

['2', '1', '0']


In [4]:
def isEipal(seq, i):
    """Checks if a string seq is an E_i palindrome."""
    ei = Ei(i)
    l = len(seq)
    if l == 1:
        if seq == str(i):
            return True
        else:
            return False
    for x in range(0, math.ceil(l/2)):
        if seq[x] != ei[int(seq[l-1-x])]:
            return False
    return(True)

In [5]:
# Example:
print(isEipal("012", 1))
print(isEipal("002", 1))
print(isEipal("01201", 2))

True
False
True


In [6]:
def testPalindromicity(seq):
    """Checks if a seq is an palindrome or and E-palindrome and 
    returns its nature."""
    if isEipal(seq,0):
        return [True, "0"]
    elif isEipal(seq, 1):
        return [True, "1"]
    elif isEipal(seq, 2):
        return [True, "2"]
    elif gpc.isPal(seq):
        return [True, "R"]
    else:
        return [False]

In [7]:
# Example:
print(testPalindromicity("0210"))
print(testPalindromicity("00")) # We want 00 to be an E_0-palindrome
print(testPalindromicity("010"))
print(testPalindromicity("02110"))

[True, '0']
[True, '0']
[True, 'R']
[False]


In [8]:
def makeEipalClosure (seq, i):
    """Makes E_i-th palindromic closure of a string."""
    ei = Ei(i)
    if isEipal(seq, i) == True:
        return(seq)
    j = 1
    while isEipal(seq[j:], i) != True:
        j = j+1
    gpc.verboseprint(2, "    {0} longest palindromic \
                     suffix : {1}".format(seq,seq[j:]))
    closure = seq
    pref = seq[j-1::-1]
    for letter in pref:
        closure = closure + ei[int(letter)]
    return(closure)

In [9]:
# Example:
print(makeEipalClosure("01", 1))
print(makeEipalClosure("00", 2))
print(makeEipalClosure("00", 0))
print(makeEipalClosure("021210", 2))

012
0011
00
0212102021


In [10]:
def make012Word(delta, theta, steps, seed = ""):
    """Makes a GPS over {0,1,2} from sequences delta and theta."""
    w = seed
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = zpu.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        gpc.verboseprint(1, "w{0} = {1}".format(step+1,w))
    return(w)

In [11]:
# Example:
make012Word("0012", "0020", 4)

'00112200'

## Naive function for checking normalization

In [12]:
def is012NormalizedNaive(delta, theta, steps):
    """Checks if delta and theta are normalized and if not, 
    returns the beginning of the normalized sequence."""
    w = ""
    l=1
    prefixes = []
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = gpc.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        prefixes.append(w)
    gpc.verboseprint(1, "Prefixes from (delta, theta): " + str(prefixes))
    gpc.verboseprint(1, "Obtained word: " + w)
    newdelta = delta[0]
    newtheta = ""
    while l <= len(w):
        prefix = w[:l]
        res = testPalindromicity(prefix)
        if res[0] == True:
            print(prefix)
            if l < len(w):
                newdelta = newdelta + w[l]
            newtheta = newtheta + res[1]           
        l=l+1
    if newdelta == delta[:steps] and newtheta == theta[:steps]:
        return [True, newdelta, newtheta]
    else:
        return [False, newdelta, newtheta]
    
    # The length of newdelta and newtheta are the same since the whole word 
    # is an palindrome because if was generated by the GPS construction
    # from delta and theta.

In [13]:
# Example: 
print(is012NormalizedNaive("0012", "0020", 4))
print(is012NormalizedNaive("00120", "00210", 4))

0
00
0011
001122
00112200
[False, '00120', '00210']
0
00
0011
001122
[True, '0012', '0021']


## Implementation of the normalization algorithm

### Preprocessing of the normalized bi-sequence for our implementation

In our implementation of the algorithm, we decided to work only with words
that has 0 as first letter, 1 at second and 2 at third in order to make the 
algorithm easier to read and write. Of course, we want it working for all
the bi-sequences, so we have to preprocess the bi-sequence so that the word
obtained has first 0, then 1 and then 2. After the result of the algorithm,
we will go back to the original letters order.

In [14]:
def substitute(dic, seq):
    """Substitutes letters in a word according to rules in dic, if there is
    no rule for the letter, keeps the letter."""
    newseq = ""
    for l in seq:
        if l in dic:
            newseq = newseq + dic[l]
        else:
            newseq = newseq + l
    return newseq

In [15]:
def compose(subs1, subs2):
    """Composes two substitutions of letter."""
    csub = {}
    for l in ["0", "1", "2"]:
        if l in subs1:
            csub[l] = subs1[l]
            if csub[l] in subs2:
                csub[l] = subs2[csub[l]]
        elif l in subs2:
            csub[l] = subs2[l]
    return csub

In [16]:
# Example:
print(compose({}, {"1": "0", "0": "1"}))
print(compose({"0": "1", "1": "0"}, {"1" : "2", "2" : "0", "0": "1"}))

{'1': '0', '0': '1'}
{'2': '0', '1': '1', '0': '2'}


In [17]:
def changeLettersOrder(delta, theta):
    """ Change (delta, theta) so that the word obtained is the same at the 
    original one, but the first symbol is 0, the second 1 and the third 2."""
    subs = {}
    subs2 = {"2": "1", "1": "2"}
    if delta[0] != "0":
        subs = {delta[0]: "0", "0": delta[0]}
        delta = substitute(subs, delta)
        theta = substitute(subs, theta)
    i = 0
    l = len(delta)
    while i < l and delta[i] == "0":
        if theta[i] == "2":
            return [delta, theta, subs]
        if theta[i] == "1":
            delta = substitute(subs2, delta)
            theta = substitute(subs2, theta)            
            return [delta, theta, compose(subs, subs2)]
        #otherwise whe have to continue
        i = i + 1
    if i < l and delta[i] == "2":
        delta = substitute(subs2, delta)
        theta = substitute(subs2, theta) 
        return [delta, theta, compose(subs, subs2)]
    return [delta, theta, subs]

In [18]:
def changeLettersOrderBack(delta, theta, subs):
    backsubs = {v:k for k,v in subs.items()}
    delta = substitute(backsubs, delta)
    theta = substitute(backsubs, theta)
    return [delta, theta]

In [19]:
# Example: 
ex1 = changeLettersOrder("111122220000", "11111RR00112")
print(ex1)
print(changeLettersOrderBack(ex1[0], ex1[1], ex1[2]), "\n")
ex2 = changeLettersOrder("000", "001")
print(ex2)
print(changeLettersOrderBack(ex2[0], ex2[1], ex2[2]), "\n")
ex3 = changeLettersOrder("011", "02")
print(ex3)
print(changeLettersOrderBack(ex3[0], ex3[1], ex3[2]), "\n")
ex4 = changeLettersOrder("0002", "0RR0")
print(ex4)
print(changeLettersOrderBack(ex4[0], ex4[1], ex4[2]), "\n")
ex5 = changeLettersOrder("111120", "1RR10")
print(ex5)
print(changeLettersOrderBack(ex5[0], ex5[1], ex5[2]))

['000011112222', '00000RR22001', {'2': '1', '1': '0', '0': '2'}]
['111122220000', '11111RR00112'] 

['000', '002', {'2': '1', '1': '2'}]
['000', '001'] 

['011', '02', {}]
['011', '02'] 

['0001', '0RR0', {'2': '1', '1': '2'}]
['0002', '0RR0'] 

['000012', '0RR02', {'2': '1', '1': '0', '0': '2'}]
['111120', '1RR10']


### Preprocessing of the beginning of the bi-sequence

We want that every word beginning with $i^l$ (where $l$ is the largest
possible) has a bi-directive sequence $(\Delta, \Theta)$ where the prefix
of $\Theta$ of lenght $l$ is equal to $E^l_i$. (We only solve the cases
when there is an $R$ instead of $E_0$ (e.g. $(0000, RE_0RE_0) \to (0000, E_0E_0E_0E_0)$, since the other cases are then solved
within the normalization process.

In [20]:
def initialNormalization(delta, theta):
    biseq = gpc.makeBiseq(delta, theta)
    m = re.match("(0(R|0))+", biseq)
    if m:
        biseq = "00"*int((m.end()-m.start())/2) + biseq[m.end():]
    return gpc.parseBiseq(biseq)

In [21]:
initialNormalization("000011", "R0R021")

['000011', '000021']

### Auxiliary functions

<img src="prefixesrules.jpg",width=600>
    
    
    

In [22]:
# List: bad prefix regex --> the new symbols instead of the last one
bad_prefixes = [
    ["(00)*02", "0012"], # 1.
    ["0012(0R12)*0R10", "1220"], # 2.
    ["0012(0R12)+01", "0R21"],  # 3.
    ["00121121", "2001"], # 4.
    ["001210", "1120"], # 5.
    ["001212", "1R02"], # 6.
    ["0012211R(111R)*1112", "1R02"], # 7.
    ["0012211R(111R)+10", "1100"], # 8.
    ["001222", "210012"], # 10.
    ["0011", "1221"], # 11.
    ["0012(0R12)*00", "1R20"], # 12. opraveno
    ["0012(0R12)*0R11", "1221"], # 13.
    ["(001221)+00122R", "211R"], # 14.
    ["(001221)+001R","120R"], # 15.
    ["(001221)+0R","002R"], # 16.
    ["(001221)+1R2R", "222R"], # 17.
    ["(001221)+00120R2R", "201R"], # 18.
    ["(001221)+002R2R", "210R"], # 19.
    ["00(120R)+122111", "1R11"], # 20.
    ["0012(0R12)+0R2022", "2112"], # 21.
    ["(00)+1212", "1R02"], # 22.
    ["0012(0R12)+2020", "2R10"], # 23.
    ["(00)+1210", "1120"], # 24.
    ["0012(0R12)+0R2112", "1022"], # 25.
    ["0012211R(111R)+110021", "2R10"], # 26.
    ["001221(1R11)+1R2201", "0021"] # 27.
]
# Special case of prefix 9: 
# bad prefix regex --> the new symbols instead of the last two
badprefix9 = ["011R", "00120R"] # 9. !! the last two letters are rewritten !!

In [33]:
a_b2 = [i[0]+i[1] for i in itertools.product('012', repeat = 2)]
i = ["0", "1", "2"]
rules1 = [ [k[0][0]+Ei(k[1])[int(k[0][1])]+k[0][1], "R"+k[1]+k[1]] for k in itertools.product(a_b2, i)]
rules2 = [ [k[0][0]+Ei(k[1])[int(k[0][1])]+k[0][1], k[1]+"RR"] for k in itertools.product(a_b2, i)]
rules1 = [ [k[0][0]+Ei(k[1])[int(k[0][1])]+k[0][1], "R"+k[1]+k[1]] for k in itertools.product(a_b2, i)]
print(rules1)

[['000', 'R00'], ['020', 'R11'], ['010', 'R22'], ['021', 'R00'], ['011', 'R11'], ['001', 'R22'], ['012', 'R00'], ['002', 'R11'], ['022', 'R22'], ['100', 'R00'], ['120', 'R11'], ['110', 'R22'], ['121', 'R00'], ['111', 'R11'], ['101', 'R22'], ['112', 'R00'], ['102', 'R11'], ['122', 'R22'], ['200', 'R00'], ['220', 'R11'], ['210', 'R22'], ['221', 'R00'], ['211', 'R11'], ['201', 'R22'], ['212', 'R00'], ['202', 'R11'], ['222', 'R22']]


In [148]:
def findNextBadFactor(biseq):
    #['0'+ ''.join(i) for i in itertools.product('01', repeat=rep1)]


SyntaxError: unexpected EOF while parsing (<ipython-input-148-f62f02018dc5>, line 2)

In [61]:
print([i[1]+Ei() for i in itertools. product('012', repeat = 2)])
[i for i in itertools. product('012', repeat = 2)]

['0', '1', '2', '0', '1', '2', '0', '1', '2']


[('0', '0'),
 ('0', '1'),
 ('0', '2'),
 ('1', '0'),
 ('1', '1'),
 ('1', '2'),
 ('2', '0'),
 ('2', '1'),
 ('2', '2')]

In [53]:
def isNormalized(biseq):
    matches = []
    # Looking for bad prefixes
    for prefixRule in bad_prefixes:
        match = re.match(prefixRule[0], biseq)
        if match:
            index = match.end() - 2
            matches.append([index, prefixRule[1], 2])
            # the third number is the length of the sequence we replace
            
    # Special case of prefix 9
    match = re.match(badprefix9[0], biseq)
    if match:
        index = match.end() - 4
        matches.append([index, prefixRule[1], 4])
    # Final "leading" prefix    
    final = []
    if matches:
        final = matches[0]
        for rule in matches[1:]:
            if rule[0] < final[0]:
                final = rule
                return final
                
    # Looking for bad factors if there is no bad prefix
    findNextBadFactor()
    
    
    # Rule to apply: choosing smallest index in the word
    #print(matches)
    
    return final

In [24]:
def applyRule(biseq, rule):
    return biseq[:rule[0]] + rule[1] + biseq[rule[0] + rule[2]:]

### Normalization algorithm

In [25]:
def normalize(delta, theta):
    """Returns the normalized directive bi-sequence giving the same GPS word
    as (delta, theta)"""
    # Normalization of the letters order
    [delta, theta, substitution] = changeLettersOrder(delta, theta)
    
    # Normalization of the prefix
    [delta, theta] = initialNormalization(delta, theta)
    
    # The main algorithm
    biseq = gpc.makeBiseq(delta, theta)
    applicableRule = []
    applicableRule = isNormalized(biseq)    
    while applicableRule:
        biseq = applyRule(biseq, applicableRule);
        applicableRule = isNormalized(biseq)
    
    [delta, theta] = gpc.parseBiseq(biseq)
    print(delta, theta)
    return [changeLettersOrderBack(delta, theta, substitution)]

In [26]:
r = normalize("010", "020")

0112 02R0


In [27]:
is012NormalizedNaive("011", "021", 3)

0
01
0112


[True, '011', '021']

In [28]:
gpc.verbose =1

In [37]:
is012NormalizedNaive("0102110", "02R0121", 7)

0
01
010
01020
01020120212
010201202121020212012101
0102012021210202120121010212101201020
01020120212102021201210102121012010202101020120212


[False, '01021102', '02R01201']

In [52]:
re.search("abb", "000abbabb")

SyntaxError: invalid syntax (<ipython-input-52-c05e46e02805>, line 1)

In [55]:
"wwweeee"[:2]

'ww'